In [ ]:
#Import libraries 
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas as hvplot
import pandas as pd 
import requests
import csv

#Define the file path
file = "./Data/time_series_covid19_confirmed_US.csv"
#Read the csv
covid_df = pd.read_csv(file)
covid_df.head()

In [ ]:
#Cleaning the data
clean_df = covid_df.loc[(covid_df["Long_"] != 0.0) & (covid_df["Lat"] != 0.0)]
clean_df = clean_df.dropna()

clean_df.head()

In [ ]:
#Example
minnesota_df = clean_df.loc[covid_df["Province_State"] == "Minnesota"]
minnesota_df.head()

In [ ]:
minnesota_map = minnesota_df.hvplot.points(
    "Long_",
    "Lat",
    geo = True,
    color="Admin2"
    
)
minnesota_map

In [ ]:
minnesota_df.plot("Admin2","2/28/23",kind="bar",figsize=(30,10))


In [ ]:
#Cleaning code

In [ ]:
#Viktor

In [ ]:
#Viktor

In [ ]:
#Samantha 
#display csv
df_bachelor= pd.read_csv('./Education/bachelor_degree.csv')
df_bachelor= df_bachelor.loc[df_bachelor["County"] != "United States"]
df_bachelor = df_bachelor.loc[df_bachelor["County"] != "Minnesota"]
print("Bachelor Degree Data:")
df_bachelor

In [ ]:
df_high_school = pd.read_csv('./Education/high school.csv')
df_high_school = df_high_school.loc[df_high_school["County"] != "United States"]
df_high_school = df_high_school.loc[df_high_school["County"] != "Minnesota"]
print("\nHigh School Education Data:")
df_high_school

In [ ]:
df_9th_grade = pd.read_csv('./Education/9th grade.csv')
df_9th_grade = df_9th_grade.loc[df_9th_grade["County"] != "United States"]
df_9th_grade = df_9th_grade.loc[df_9th_grade["County"] != "Minnesota"]
print("\n9th Grade Education Data:")
df_9th_grade

In [ ]:
# Merge the DataFrames on the 'County' column
merged_df = df_bachelor.merge(df_high_school, on='County', suffixes=('_bachelor', '_high_school'))
merged_df = merged_df.merge(df_9th_grade, on='County')

# Rename columns for clarity if needed
merged_df.rename(columns={
    'Value Bachelor (Percent)': 'Percent_Bachelor',
    'Value High School (Percent)': 'People_High_School',
    'Value 9th Grade (Percent)': 'Percent_9th_Grade'
}, inplace=True)

# Print the merged DataFrame
print("\nMerged Data:")
merged_df

In [ ]:
# Update column names in the code based on the actual column names
counties = merged_df['County']
bachelor_degrees = merged_df['Value Bachelor(Percent)']  # Check exact match
high_school_education = merged_df['Value High School(Percent)']  # Check exact match
ninth_grade_education = merged_df['Value 9th Grade(Percent)']  # Check exact match
merged_df.columns = merged_df.columns.str.strip()
# Plotting
plt.figure(figsize=(15, 15))

# Stacked bar chart
plt.barh(counties, bachelor_degrees, color='lightblue', label='Bachelor Degree')
plt.barh(counties, high_school_education, left=bachelor_degrees, color='lightgreen', label='High School Education')
plt.barh(counties, ninth_grade_education, left=bachelor_degrees + high_school_education, color='lightcoral', label='9th Grade Education')

# Adding labels and title
plt.xlabel('Percent of Population')
plt.ylabel('County')
plt.title('Education Levels by County')
plt.legend(title='Education Level')
plt.gca().invert_yaxis()  # Invert y-axis to have the highest values at the top

# Show the plot
plt.show()

In [ ]:
#display csv
df_census= pd.read_csv('./Education/Census Data.csv')
print("\nCensus Data:")
df_census

In [ ]:
# Extract columns
years = df_census['year']
high_school_graduate = df_census['high_school_graduate']
high_school_grad_or_higher = df_census['high_school_graduate_or_higher']  # Correct column name
bachelors_degree = df_census['bachelors_degree']
bachelors_degree_or_higher = df_census['bachelors_degree_or_higher']

# Create the plot
plt.figure(figsize=(10, 6))

plt.plot(years, high_school_graduate, marker='o', label='High School Graduate')
plt.plot(years, high_school_grad_or_higher, marker='o', label='High School Graduate or Higher')
plt.plot(years, bachelors_degree, marker='o', label='Bachelors Degree')
plt.plot(years, bachelors_degree_or_higher, marker='o', label='Bachelors Degree or Higher')

# Adding titles and labels
plt.title('Educational Attainment Over Years by Population')
plt.xlabel('Year')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.xticks(years)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Calculate percentages
df_census['percent_high_school_graduate'] = (df_census['high_school_graduate'] / df_census['population_25_years_and_over']) * 100
df_census['percent_high_school_graduate_or_higher'] = (df_census['high_school_graduate_or_higher'] / df_census['population_25_years_and_over']) * 100
df_census['percent_bachelors_degree'] = (df_census['bachelors_degree'] / df_census['population_25_years_and_over']) * 100
df_census['percent_bachelors_degree_or_higher'] = (df_census['bachelors_degree_or_higher'] / df_census['population_25_years_and_over']) * 100

# Display the updated DataFrame with percentages
print("\nUpdated Census Data with Percentages:")
print(df_census[['year', 'percent_high_school_graduate', 'percent_high_school_graduate_or_higher',
                 'percent_bachelors_degree', 'percent_bachelors_degree_or_higher']])

In [ ]:
# Set the figure size
plt.figure(figsize=(12, 8))

# Bar width and positions
bar_width = 0.2
index = range(len(df_census['year']))

# Plot bars for each category
plt.bar([i - 1.5 * bar_width for i in index], df_census['percent_high_school_graduate'], bar_width, label='High School Graduate')
plt.bar([i - 0.5 * bar_width for i in index], df_census['percent_high_school_graduate_or_higher'], bar_width, label='High School Graduate or Higher')
plt.bar([i + 0.5 * bar_width for i in index], df_census['percent_bachelors_degree'], bar_width, label='Bachelors Degree')
plt.bar([i + 1.5 * bar_width for i in index], df_census['percent_bachelors_degree_or_higher'], bar_width, label='Bachelors Degree or Higher')

# Adding titles and labels
plt.title('Educational Attainment Percentages Over Years')
plt.xlabel('Year')
plt.ylabel('Percentage (%)')
plt.xticks(index, df_census['year'])
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
df_years= pd.read_csv('./Education/2019^02021.csv')
df_years.head()

In [ ]:
# Correlation between high school or higher attainment and population of 18-24 year olds
correlation_high_school_population = df_years[['high_school_or_equal_2019', 'population_18-24_2019']].corr().iloc[0, 1]
correlation_bachelors_population = df_years[['bachelors_or_higher_2019', 'population_18-24_2019']].corr().iloc[0, 1]
correlation_high_school_population_2 = df_years[['high_school_or_equal_2021', 'population_18-24_2021']].corr().iloc[0, 1]
correlation_bachelors_population_2 = df_years[['bachelors_or_higher_2021', 'population_18-24_2021']].corr().iloc[0, 1]

print(f"Correlation between High School or Higher Attainment and Population 18-24 (2019): {correlation_high_school_population}")
print(f"Correlation between Bachelor's or Higher Attainment and Population 18-24 (2019): {correlation_bachelors_population}")
print(f"Correlation between High School or Higher Attainment and Population 18-24 (2021): {correlation_high_school_population_2}")
print(f"Correlation between Bachelor's or Higher Attainment and Population 18-24 (2021): {correlation_bachelors_population_2}")

In [ ]:
# Calculate percentages
df_years['hs_percent_2019'] = (df_years['high_school_or_equal_2019'] / df_years['population_18-24_2019']) * 100
df_years['bachelor_percent_2019'] = (df_years['bachelors_or_higher_2019'] / df_years['population_18-24_2019']) * 100
df_years['hs_percent_2021'] = (df_years['high_school_or_equal_2021'] / df_years['population_18-24_2021']) * 100
df_years['bachelor_percent_2021'] = (df_years['bachelors_or_higher_2021'] / df_years['population_18-24_2021']) * 100

# Display the DataFrame with the calculated percentages
print(df_years[['states', 'hs_percent_2019', 'bachelor_percent_2019', 'hs_percent_2021', 'bachelor_percent_2021']])

# Optionally, save the results to a new CSV file
df_years.to_csv('education_percentages.csv', index=False)

In [ ]:
# Plotting the line graphs
plt.figure(figsize=(14, 7))

# Line plot for high school or higher percentage
plt.plot(df_years['states'], df_years['hs_percent_2019'], marker='o', label='High School or Higher 2019')
plt.plot(df_years['states'], df_years['hs_percent_2021'], marker='o', linestyle='--', label='High School or Higher 2021')

# Line plot for bachelor's or higher percentage
plt.plot(df_years['states'], df_years['bachelor_percent_2019'], marker='s', label='Bachelor\'s or Higher 2019')
plt.plot(df_years['states'], df_years['bachelor_percent_2021'], marker='s', linestyle='--', label='Bachelor\'s or Higher 2021')

# Adding labels and title
plt.xlabel('States')
plt.ylabel('Percentage (%)')
plt.title('Percentage of Educational Attainment by Year for Ages 18-24')
plt.xticks(rotation=90)  # Rotate state names for better visibility
plt.legend()
plt.grid(True)

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:
# Create a figure and axis
plt.figure(figsize=(14, 7))

# Define the position of bars on the x-axis
bar_width = 0.35
index = np.arange(len(df_years['states']))

# Bar plot for high school or higher percentage
plt.bar(index - bar_width/2, df_years['hs_percent_2019'], bar_width, label='High School or Equal 2019', color= 'green')
plt.bar(index + bar_width/2, df_years['hs_percent_2021'], bar_width, label='High School or Equal 2021', color= 'black')

# Bar plot for bachelor's or higher percentage
plt.bar(index - bar_width/2, df_years['bachelor_percent_2019'], bar_width, label='Bachelor\'s or Higher 2019', alpha=0.5, color= 'blue')
plt.bar(index + bar_width/2, df_years['bachelor_percent_2021'], bar_width, label='Bachelor\'s or Higher 2021', alpha=0.5, color= 'magenta')

# Adding labels and title
plt.xlabel('States')
plt.ylabel('Percentage (%)')
plt.title('Percentage of Educational Attainment by Year for Ages 18-24')
plt.xticks(index, df_years['states'], rotation=90)  # Rotate state names for better visibility
plt.legend()
plt.grid(True)

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
# Create a figure and axis
plt.figure(figsize=(14, 7))

# Define the position of bars on the x-axis
bar_width = 0.35
index = np.arange(len(df_years['states']))

# Bar plot for bachelor's or higher percentage
plt.bar(index - bar_width/2, df_years['bachelor_percent_2019'], bar_width, label='Bachelor\'s or Higher 2019', alpha=0.5, color= 'blue')
plt.bar(index + bar_width/2, df_years['bachelor_percent_2021'], bar_width, label='Bachelor\'s or Higher 2021', alpha=0.5, color= 'magenta')

# Adding labels and title
plt.xlabel('States')
plt.ylabel('Percentage (%)')
plt.title('Percentage of Educational Attainment by Year for Ages 18-24')
plt.xticks(index, df_years['states'], rotation=90)  # Rotate state names for better visibility
plt.legend()
plt.grid(True)

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
# Create a figure and axis
plt.figure(figsize=(14, 7))

# Define the position of bars on the x-axis
bar_width = 0.35
index = np.arange(len(df_years['states']))

# Bar plot for high school or higher percentage
plt.bar(index - bar_width/2, df_years['hs_percent_2019'], bar_width, label='High School or Equal 2019', color= 'green')
plt.bar(index + bar_width/2, df_years['hs_percent_2021'], bar_width, label='High School or Equal 2021', color= 'black')

# Adding labels and title
plt.xlabel('States')
plt.ylabel('Percentage (%)')
plt.title('Percentage of Educational Attainment by Year for Ages 18-24')
plt.xticks(index, df_years['states'], rotation=90)  # Rotate state names for better visibility
plt.legend()
plt.grid(True)

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
#Samantha

In [ ]:
#Nicholas

In [ ]:
#Nicholas

In [ ]:
#Mysee

In [ ]:
#Mysee

In [ ]:
#Micah

In [ ]:
#Micah